# Introduction

This notebook partitions Sentinel‑2 RGB imagery into 256×256 patches and generates aligned image–mask pairs using ESA WorldCover masks via the Microsoft Planetary Computer STAC API. Key steps covered:
- search and download suitable Sentinel‑2 and WorldCover assets for a given bbox/year.
- reproject/resample WorldCover to the Sentinel grid (nearest for categorical masks).
- verify alignment (size, CRS, transform) and tile into image–mask pairs.
- save patches for training or publishing.

### Imports 

In [ ]:
%pip install torchgeo
%matplotlib inline

In [ ]:
import timm
from torchgeo.models import ResNet18_Weights, ResNet50_Weights, resnet18, resnet50
from pathlib import Path
import sys

In [ ]:
# Code written with ChatGPT-5
def find_repo_root(start: Path = Path.cwd(), markers=("pyproject.toml", "setup.py", ".git", "README.md")) -> Path:
    """_summary_
    Set the folder to the current path for correct folder alignment. 
    """
    p = start.resolve()
    while True:
        if any((p / m).exists() for m in markers):
            return p
        if p.parent == p: 
            return start.resolve()
        p = p.parent

repo_root = find_repo_root()
sys.path.insert(0, str(repo_root))

print("repo_root:", repo_root)

### Load dataset
We use Microsoft Planetary Computer Stack as the backbone API to collect image pairs. 

In [ ]:
# 1. Connect to Microsoft Planetary Computer STAC Catalog
import pystac_client
import planetary_computer
from planetary_computer import sign_url
from shapely.geometry import box, mapping

# Variables specifying the position, year and crop sizes
bbox = [10.65, 59.85, 10.85, 59.95]
geom = mapping(box(bbox[0], bbox[1], bbox[2], bbox[3]))
year = 2021
crop_size = 256

# https://pystac-client.readthedocs.io/en/stable/usage.html#loading-data
# Load the Microsoft Stac client
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace
)

#### Masks
We use the masks generated by ESA WorldCover. This is one of the biggest generated segmentation maps, and has been widely used due to it's good precision and simplicity. 

In [ ]:
# --- ESA WorldCover ---
print("ESA world cover")
search_mask = catalog.search(
    max_items = 1,
    intersects = geom,
    collections=["esa-worldcover"],
    datetime=f"{year}-01-01/{year}-12-31"
)
mask_items = search_mask.item_collection()

if len(mask_items) == 0:
    raise ValueError("No WorldCover data found for this location.")

selected_mask_item = mask_items[0]

asset_items = selected_mask_item.assets.items()

asset_key = "map" # Get the full rgb image directly
href = selected_mask_item.assets[asset_key].href
signed_hrf = sign_url(href)

ESA world cover


#### Images
The images are taken from raw images taken by the Sentinel-2 satellite. We select the RGB bands and filter for 10% cloud-coverage to find the best raster's within the timespan selected. 

In [4]:
from src.data.patchify_data import save_as_images
# --- Sentinel-2 ---
print("Sentinel2")
search_s2 = catalog.search(
    max_items = 1,
    collections=["sentinel-2-l2a"],
    intersects=geom,
    datetime=f"{year}-01-01/{year}-12-31",
    query={"eo:cloud_cover": {"lt": 10}}, # Less than 10% clouds
    sortby=[{'field': 'eo:cloud_cover', 'direction': 'asc'}] # Get the clearest one
)
s2_items = search_s2.item_collection()

if len(s2_items) == 0:
    raise ValueError("No clear Sentinel-2 images found.")
    
selected_s2_item = s2_items[0]
asset_items = selected_s2_item.assets.items()

asset_key = "visual" # Get the full rgb image directly
href = selected_s2_item.assets[asset_key].href
signed_hrf = sign_url(href)

#save_as_images(signed_hrf, bands=(1,2,3), img_size=256, img_prefix="sentinel2", out_directory="sentinel_patches")

Sentinel2


#### Inspect elements
We perform a simple size check to see if the patches align in size. This is crucial to get accurate image-mask pairs for image segmentation. 

In [ ]:
from math import ceil
import rasterio
from planetary_computer import sign_url

mask_href = sign_url(selected_mask_item.assets["map"].href)
s2_href   = sign_url(selected_s2_item.assets["visual"].href)

for name, href in [("worldcover", mask_href), ("sentinel2", s2_href)]:
    with rasterio.open(href) as src:
        w,h = src.width, src.height
        print(name, "width,height:", w, h)
        tile = 256
        ncols = ceil(w / tile)
        nrows = ceil(h / tile)
        print("  tiles (ceil):", nrows, "x", ncols, "=", nrows*ncols)
        print("  dtype,count:", src.dtypes, src.count)

worldcover width,height: 36000 36000
  tiles (ceil): 141 x 141 = 19881
  dtype,count: ('uint8',) 1
sentinel2 width,height: 10980 10980
  tiles (ceil): 43 x 43 = 1849
  dtype,count: ('uint8', 'uint8', 'uint8') 3


#### Preprocessing with transformation
Using rasterio leads to different trasnforms due to their type. We therefore need to either transform the images or use odc.stac.load. Here we use rasterio transform as this module is used in other parts of the project. 

In [ ]:
import tempfile
import rasterio
from rasterio.vrt import WarpedVRT
from rasterio.enums import Resampling
from planetary_computer import sign_url
import numpy.testing as npt

from src.data.patchify_data import save_as_mask_pairs

# TODO: add this to utils
def align_mask_to_reference(mask_href, ref_href, out_path=None):
    """_summary_
    Reproject/resample single-band categorical mask (mask_href) to the grid of ref_href.
    Returns local GeoTIFF path you can pass to save_as_mask unchanged.
    """
    if out_path is None:
        tmp = tempfile.NamedTemporaryFile(suffix=".tif", delete=False)
        out_path = tmp.name
        tmp.close()

    with rasterio.open(ref_href) as ref:
        profile = ref.profile.copy()
        # single-band categorical mask
        profile.update(driver="GTiff", count=1, dtype="uint8", compress="deflate", tiled=True)

        with rasterio.open(mask_href) as src:
            with WarpedVRT(
                src,
                crs=ref.crs,
                transform=ref.transform,
                width=ref.width,
                height=ref.height,
                resampling=Resampling.nearest,
            ) as vrt:
                with rasterio.open(out_path, "w", **profile) as dst:
                    dst.write(vrt.read(1), 1)

    return out_path

mask_href = selected_mask_item.assets["map"].href
s2_href   = selected_s2_item.assets["visual"].href

aligned_mask_tif = align_mask_to_reference(sign_url(mask_href), sign_url(s2_href))

#### Save as pairs
We now have aligned image-mask pairs that have been correctly preprocessed. The last step is to save the images and perform a simple check that the pairs align. 

In [ ]:
# TODO: create this as a test! (written with ChatGPT-5)
with rasterio.open(sign_url(s2_href)) as s2, rasterio.open(aligned_mask_tif) as m:
    assert (s2.width, s2.height) == (m.width, m.height), \
        f"size mismatch: s2={s2.width}x{s2.height}, mask={m.width}x{m.height}"
    assert s2.crs == m.crs, f"CRS mismatch: s2={s2.crs}, mask={m.crs}"
    # transforms are Affine; allow tiny numerical tolerance
    npt.assert_allclose(tuple(s2.transform), tuple(m.transform), atol=1e-6, rtol=0)
    assert m.count == 1, f"mask has {m.count} bands (expected 1)"
    print("PASS: aligned_mask_tif matches Sentinel-2 grid (width,height,crs,transform)")

save_as_mask_pairs(aligned_mask_tif, sign_url(s2_href))